# CSV Data
For both this part and the MySQL part below, the data was explored per column, either by printing column, using the function `pd.unique()`, or converting it to categorical variables and analysing the result. This was an easy way to find anomalies in the data and see what needed to be fixed. For dates and numerical values, I counted `NaN`s and `NaT`s using `isna()`.

In [57]:
import pandas as pd 

# read from csv
df = pd.read_csv("BSCY4.csv", sep=",")

## Handling the Date column
The Date column combines dates in three different formats:
- `%Y-%m-%d`
- `%d-%m-%Y` 
- `%d/%m` (where year must be fetched from 'year' column)

To achieve consistency, all rows will be formatted according to the first type, as it contains the majority of the rows in the dataset. This is done by finding the indices of the rows with differently formatted dates, applying helper functions to format them, saving them to a new column called `temp`, and then combing this column with the original date column. 

In [58]:
# helper function to reorder dates of type d-m-Y
# by switching year and day 
def order_date(d):
    d[0], d[-1] = d[-1], d[0]
    d = "-".join(d)
    return d

# helper function to reformat dates of type d/m by getting
# the year and calling the order_date function
def get_year(d, index):
    year = df['year'][index]
    d.append(str(year).rstrip('0').rstrip('.'))
    d = order_date(d)
    return d

In [59]:
# find rows with wrongly formatted dates. there are 338 in total.
dates = pd.to_datetime(df['Date'], format="%Y-%m-%d", errors="coerce")
print("Abnormalities in date:", dates.isna().sum())

# get row indices 
# df.loc['Date']
nat_indices = dates[dates.isnull() == True].index

# create a new temporary column, then populate 
# it with adjusted date strings
df['temp'] = pd.NaT

for index in nat_indices:
    date = df['Date'][index]
    d = date.split('-')
    
    if len(d[-1]) == 4:
        d = order_date(d)
        df.at[index, 'temp'] = d
    else:
        d = date.split('/')
        d = get_year(d, index)
        df.at[index, 'temp'] = d

# merge the date column with the temporary
# column 
df['Date'] = dates
df['Date'] = df['Date'].fillna(df['temp'])
df.drop('temp', axis=1, inplace=True)

Abnormalities in date: 338


## Handling the Type column
The Type column is categorical and contains 2 different types:
- conventional
- organic or Org.

For consistency, any rows with 'Org.' will be reformatted to 'organic'. 

In [60]:
# replace 'Org.' with 'organic', then convert column
# to type category. this gives us 2 categories in total.
print("\nUnique categories in 'type' (before cleaning):", df['type'].unique())
print("Abnormalities in 'type':", df['type'].value_counts().to_dict()['Org.'])
df['type'] = df['type'].astype(str).str.replace('Org.','organic')
df['type'] = df['type'].astype('category')


Unique categories in 'type' (before cleaning): ['conventional' 'organic' 'Org.']
Abnormalities in 'type': 169


## Handling the Average Price column
The AveragePrice column is numeric and contains:
- numbers with commas (e.g. 2,4)
- numbers with decimal points (e.g. 2.4)
- legitimate NaNs

For consistency, commas will be replaced with decimal points and the NaNs will be kept (20 in total).

In [39]:
# replace commas with decimal points and cast to numeric 
errors = pd.to_numeric(df['AveragePrice'], errors='coerce')
df['AveragePrice'] = df['AveragePrice'].astype(str).str.replace(',','.')
df['AveragePrice'] = pd.to_numeric(df['AveragePrice'], errors="coerce")

# 20 legitimate nans in total
print("Legitimate NaNs in average price:", df['AveragePrice'].isna().sum())
print("Abnormalities in average price:", errors.isna().sum() - df['AveragePrice'].isna().sum())

Legitimate NaNs in average price: 20
Abnormalities in average price: 30


# MySQL Data
A `pymysql` connection is made to read data from the 'Avocado' table in the database 'BSCY4' on localhost.

## Handling the Region column
Region is a categorical column and contains 57 regions in total before cleaning. 
Some region names have extra white spaces or dashes signifying a metroplex (e.g. 'Baltimore-Washington', which is the overlapping labor market region of the cities of Washington and Baltimore). For consistency, all white spaces and dashes are removed, giving a total of 54 **unique** categories.

## Handling the Year column
The year column contains years which are formatted in two different ways:
- `yyyy` (e.g 2018)
- `yy` (e.g 18) 

For consistency all dates will be formatted according to the former, less ambigious, type. The years will then be converted to `int64`.

## Handling the Type column
The type column contains a single category, with labels using different capitalisation ('Conventional' and 'conventional'). These labels will be formatted to lowercase and then converted to categories. 

In [55]:
import pymysql
import numpy as np
import getpass

con = pymysql.connect(host="127.0.0.1", user="data_mining", password=getpass.getpass(prompt='Password:'), database="BSCY4")
sql = pd.read_sql("SELECT * FROM AVOCADO", con)

# clean up duplicate categories and convert region 
# to categorical type 
errors = len(sql['region'].astype('category').cat.categories)
sql['region'] = sql['region'].astype(str).str.strip().str.replace("-", "")
sql['region'] = sql['region'].astype('category')
print("Abnormalities in region:", errors - len(sql['region'].cat.categories))
print("Unique regions:", len(sql['region'].cat.categories))
print(sql['region'].cat.categories)

# fix year format and convert years to int64
errors = len([x for x in sql['year'].astype(str) if len(x) == 2])
print("\nAbnormalities in year:", errors)
sql['year'] = sql['year'].astype(str)
sql['year'] = sql.apply(lambda row: ('20' + row['year']) if len(row['year']) == 2 else row['year'], axis=1)
sql['year'] = sql['year'].astype(np.int64)

# fix capitalisation in type column and convert to
# categorical variables
print("\nUnique categories in 'type' (before cleaning):", sql['type'].unique())
print("Abnormalities in 'type':", sql['type'].value_counts().to_dict()['Conventional'])
sql['type'] = sql['type'].astype(str).str.lower()
sql['type'] = sql['type'].astype('category')

con.close()

Password:········
Abnormalities in region: 3
Unique regions: 54
Index(['Albany', 'Atlanta', 'BaltimoreWashington', 'Boise', 'Boston',
       'BuffaloRochester', 'California', 'Charlotte', 'Chicago',
       'CincinnatiDayton', 'Columbus', 'DallasFtWorth', 'Denver', 'Detroit',
       'GrandRapids', 'GreatLakes', 'HarrisburgScranton',
       'HartfordSpringfield', 'Houston', 'Indianapolis', 'Jacksonville',
       'LasVegas', 'LosAngeles', 'Louisville', 'MiamiFtLauderdale', 'Midsouth',
       'Nashville', 'NewOrleansMobile', 'NewYork', 'Northeast',
       'NorthernNewEngland', 'Orlando', 'Philadelphia', 'PhoenixTucson',
       'Pittsburgh', 'Plains', 'Portland', 'RaleighGreensboro',
       'RichmondNorfolk', 'Roanoke', 'Sacramento', 'SanDiego', 'SanFrancisco',
       'Seattle', 'SouthCarolina', 'SouthCentral', 'Southeast', 'Spokane',
       'StLouis', 'Syracuse', 'Tampa', 'TotalUS', 'West', 'WestTexNewMexico'],
      dtype='object')

Abnormalities in year: 3208

Unique categories in 'type'

# Consolidating the data 
In order to concatenate the two dataframes, the column types and names need to be checked to make sure they are consistent. After this, a `pd.concat()` can be done to stack the dataframes on top of each other. 

In [9]:
df.head()
sql.head()

# drop 'unamed' column from df, a column of the original indices  
# which is not needed
df.drop('Unnamed: 0', axis=1, inplace=True)
print("SQL columns:", len(sql.columns), "\nCSV columns:", len(df.columns))

SQL columns: 13 
CSV columns: 13


In [10]:
print("Data types for CSV data:\n", df.dtypes)
print("\nData types for SQL data:\n", sql.dtypes)

# sql date needs to be converted to datetime
sql['Date'] = pd.to_datetime(sql['Date'], format="%Y-%m-%d", errors="coerce")

# csv region + year need to be converted
df['year'] = df['year'].astype(np.int64)
df['region'] = df['region'].astype('category')

Data types for CSV data:
 Date            datetime64[ns]
AveragePrice           float64
Total Volume           float64
4046                   float64
4225                   float64
4770                   float64
Total Bags             float64
Small Bags             float64
Large Bags             float64
XLarge Bags            float64
type                  category
year                   float64
region                  object
dtype: object

Data types for SQL data:
 Date              object
AveragePrice     float64
TotalValue       float64
c4046            float64
c4225            float64
c4770            float64
TotalBags        float64
SmallBags        float64
LargeBags        float64
XLargeBags       float64
type            category
year               int64
region          category
dtype: object


In [11]:
df.head()
sql.head()

# column names for df + sql are different, need to 
# make them consistent before merging
column_names = {
    'TotalValue':'Total Volume', 
    'c4046': '4046', 
    'c4225': '4225', 
    'c4770': '4770', 
    'TotalBags': 'Total Bags',
    'SmallBags': 'Small Bags',
    'LargeBags': 'Large Bags',
    'XLargeBags': 'XLarge Bags'
}

# concatenate the data frames
new_df = pd.concat([df, sql.rename(columns=column_names)], ignore_index=True)

# need to cast the type column to categorical again. since the sql df
# only had 1 category, the concatenation results in an 'object' type
new_df['type'] = new_df['type'].astype('category')

## The final result

In [12]:
print("\nData types for concatenated data:\n", new_df.dtypes)
print("\nTotal number of columns:\n", len(new_df.columns))
print("\nTotal number of rows:\n", len(new_df.index))

new_df.to_csv("new_csv.csv")


Data types for concatenated data:
 Date            datetime64[ns]
AveragePrice           float64
Total Volume           float64
4046                   float64
4225                   float64
4770                   float64
Total Bags             float64
Small Bags             float64
Large Bags             float64
XLarge Bags            float64
type                  category
year                     int64
region                category
dtype: object

Total number of columns:
 13

Total number of rows:
 18249


In [61]:
# Alternative way of looking through year (for reference)
for i, item in df['year'].iteritems():
    if len(item) == 2: 
        df.iloc[i, df.columns.get_loc('year')] = '20' + item